In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from langgraph.graph import StateGraph, END, START
from typing import Annotated, TypedDict, List
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AnyMessage
import requests
import json

In [3]:
model = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [4]:
from tavily import TavilyClient
tavily = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [5]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

#### Agents

1) city_selection_agent :- As a City Selection Expert, your task is to analyze various travel data and choose the best city for the user, considering the current weather, season, and cost factors. Your expertise in evaluating these criteria will ensure that the selected destination offers the most favorable conditions and value for travelers.
2) local_expert :- As a Local Expert in this city, your task is to offer the best insights about the selected city. Use your extensive knowledge about its attractions and customs to provide valuable and detailed information that will enhance the travel experience for visitors.
3) travel_concierge :- As an Amazing Travel Concierge, your task is to leverage your decades of experience in travel planning to create outstanding travel itineraries. These should include detailed budget plans and packing suggestions tailored to the chosen city, ensuring a seamless and enjoyable travel experience for the user.



	1.	City Selection Expert: Imagine you’re a travel enthusiast with a knack for picking the perfect destinations. Your mission is to sift through travel data and pinpoint the best city for our users, considering the current weather, season, and budget. Your skill in evaluating these factors will help ensure that travelers enjoy the best conditions and value during their trips.
	2.	Local Expert: Picture yourself as a local guide with a deep love for your city. Your goal is to share the most insightful and interesting details about this place. Use your extensive knowledge about its attractions and customs to provide valuable information that will enrich the travel experience for visitors.
	3.	Travel Concierge: Think of yourself as a seasoned travel planner with years of experience crafting perfect trips. Your task is to create amazing travel itineraries, complete with detailed budget plans and packing tips tailored to the chosen city. Your expertise will ensure a smooth and enjoyable journey for our users.

In [6]:
class AgentState(TypedDict):
    Where_from: str
    Where_to: str
    Local_expert: str
    Hotel_details: str
    Hotel_expert: str
    Departure_date: str
    Return_date: str
    Travel_preference: str # This maybe by car, flight etc
    Travel_expert: str

In [7]:
LOCAL_EXPERT_QUERIES_PROMPT = """ You are an expert local guide of . \
You are living there for almost 15years \
Your goal is to generate two queries such that it will retrieve overview of what \
the city has to offer, including hidden gems, cultural hotspots, must-visit landmarks,\
"""

In [8]:
LOCAL_EXPERT_PROMPT = """ Your goal is to share the most insightful and interesting details about this place. \
Use the context given below of the {city} to find about its attractions and \
customs to provide valuable information that will enrich the travel experience for visitors.\



example output:- 
1) Time's Square :- This is a great place to walk around at the night times. etc
......
"""

In [9]:
def Local_expert_agent(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=LOCAL_EXPERT_QUERIES_PROMPT),
        HumanMessage(content = state['Where_to']),
    ])
    context = ""
    for q in queries.queries:
        print(q)
        response = tavily.search(query=q, max_results = 5, include_raw_content = True, include_domains = ["expedia.com"])
        for result in response['results']:
            context = "\n\n ".join(result['raw_content'])
    best_version = model.invoke([
        SystemMessage(content=LOCAL_EXPERT_PROMPT.format(city = state['Where_to'])),
        HumanMessage(content = context),
    ]).content
    return {"Local_expert": best_version}

In [10]:
HOTEL_EXPERT_QUERIES_PROMPT = """ You are great at finding hotels at a particular {city} that user would like to visit \
Based on the user Hotel Preferences.
check_in date {Departure_date} and check out date {Return_date} \
Generate a query to find the best hotel that fits the user preferences. \
"""

In [11]:
HOTEL_EXPERT_PROMPT = """ Your goal is to share the best hotels that fit the user preference. \
Use the context given below of the hotels and choose few hotels that better fit the user preference \
Also also give a single sentence reasoning, why is that hotel better. \

Here is the user preference, {hotels}

example output:- 
1) Marriot :- This is hotel is a 5 start hotel which is luxury and it is near to many of the famous attractions. \
Per night cost would be:- 200$ per room. and also give the hotel location.
Important amenities:-
1) pool
2) spa .....etc
......
"""

In [12]:
def Hotel_expert_agent(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=HOTEL_EXPERT_QUERIES_PROMPT.format(city = state['Where_to'], 
        Departure_date = state['Departure_date'], Return_date = state['Return_date'])),
        HumanMessage(content = "\n\n" + "Here is my preferences to finding a hotel "+ state['Hotel_details']),
    ])
    context = ""
    for q in queries.queries:
        print(q)
        response = tavily.search(query=q, max_results = 5, include_raw_content = True, include_domains = ["expedia.com", "booking.com"])
        for result in response['results']:
            context = "\n\n ".join(result['raw_content'])
    best_version = model.invoke([
        SystemMessage(content=HOTEL_EXPERT_PROMPT.format(hotels = state['Hotel_details'])),
        HumanMessage(content = context),
    ]).content
    return {"Hotel_expert": best_version}

In [13]:
TRAVEL_EXPERT_PROMPT_QUERY = """ You are great at finding the best and cheapest travel \
Accommodations, based on the travel date {Departure_date} and return date {Return_date} \
from {Where_from} to {Where_to}. 

your goal is to find travel and not accommodation

Generate 2-3 queries at max based on the trip details and also you goals is to find the best and cheapest prices.
"""

In [14]:
TRAVEL_EXPERT_PROMPT = """ 
Organise these Travel details In provided order below use you creativity to order the travel plans but do not change of the information
provided below by the user. {User_preference}

A list of the top 3-5 flight options.
For each flight, provide:
Departure and arrival times
Duration of the flight
Number of stopovers (if any)
Airline
Any additional relevant details (baggage included, flight change policies, etc.)
"""

In [15]:
def Travel_expert_agent(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=TRAVEL_EXPERT_PROMPT_QUERY.format(
        Departure_date = state['Departure_date'], Return_date = state['Return_date'],
        Where_from = state['Where_from'], Where_to = state['Where_to'])),
        HumanMessage(content = "\n\n" + "Here is my travel preferences "+ state['Travel_preference']),
    ])
    context = ""

    url = "https://api-ares.traversaal.ai/live/predict"
    for q in queries.queries:
        payload = { "query": [q] }
        headers = {
            "x-api-key": os.environ.get("TRAVERSAL_AI_API_CURRENT"),
            "content-type": "application/json"
        }
        response = requests.post(url, json=payload, headers=headers)
        # print(response)
        string_data = response.content.decode('utf-8')

        # Parse JSON string to a Python dictionary
        json_data = json.loads(string_data)
        context = "\n\n ".join(json_data['data']['response_text'])
    best_version = model.invoke([
        SystemMessage(content=TRAVEL_EXPERT_PROMPT.format(User_preference = state['Travel_preference'])),
        HumanMessage(content = context),
    ]).content
    return {"Travel_expert": best_version}

In [16]:
builder = StateGraph(AgentState)

In [17]:
# builder.add_node("local_guide", Local_expert_agent)
# builder.add_node("hotel_expert", Hotel_expert_agent)
builder.add_node("travel_expert", Travel_expert_agent)

In [18]:
builder.set_entry_point("travel_expert")

In [19]:
builder.add_edge("travel_expert", END)

In [20]:
graph = builder.compile()

In [21]:

for s in graph.stream({
    "Where_from": "Atlanta",
    "Where_to" : "Miami",
    "Hotel_details": "I would prefer staying in good hotels, but not too fancy and the area around the hotels must be safe",
    "Departure_date": "july 16th 2024",
    "Return_date": "july 20th 2024",
    "Travel_preference": "Any but should be the cheapest."
}):
    print(s)

In [ ]:
print(s['travel_expert']['Travel_expert'])

**Travel Itinerary: Miami (MIA) to Atlanta (ATL)**

**Flight Option 1:**
1. Departure Time: To be confirmed
2. Arrival Time: To be confirmed
3. Duration: To be confirmed
4. Stopovers: Direct flight
5. Airline: To be confirmed
6. Additional Details: Prices were available within the past 7 days and start at $27 for one-way flights and $50 for round trip, subject to change. Baggage included.

**Flight Option 2:**
1. Departure Time: To be confirmed
2. Arrival Time: To be confirmed
3. Duration: To be confirmed
4. Stopovers: Direct flight
5. Airline: To be confirmed
6. Additional Details: Option to purchase one-way tickets with different airlines, starting at $27.

**Flight Option 3:**
1. Departure Time: To be confirmed
2. Arrival Time: To be confirmed
3. Duration: To be confirmed
4. Stopovers: Direct flight
5. Airline: To be confirmed
6. Additional Details: Refund available for the difference in price if the flight is available for a cheaper price, terms apply.

Please note: Exploring Atlan